In [1]:
import os

import torch
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

os.environ["WANDB_API_KEY"] = "KEY"
os.environ["WANDB_MODE"] = 'offline'
from itertools import combinations

import clip
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import tqdm

from einops.layers.torch import Rearrange, Reduce

from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader, Dataset
import random
import csv
from torch import Tensor
import itertools
import math
import re
import numpy as np
import argparse
from torch import nn
from torch.optim import AdamW

import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
import open_clip
device = 'cuda'
model_type = 'ViT-H-14'

vlmodel, preprocess_train, feature_extractor = open_clip.create_model_and_transforms(
    'ViT-H-14', pretrained = '/media/ubuntu/sda/neuropixels/visual_decode/open_clip_pytorch_model.bin', device = device
)


/home/ubuntu/.conda/envs/visual_decoding/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def image_encoder(images, vlmodel, preprocess, batch_size):
        features = []
        for i in range(0, len(images), batch_size):
            batch_images = images[i:i+batch_size]
            batch_input = torch.stack([preprocess(img) for img in batch_images]).to('cuda')
            with torch.no_grad():
                batch_features = vlmodel.encode_image(batch_input)
                batch_features /= batch_features.norm(dim=-1, keepdim=True)
            features.append(batch_features.cpu())
        return torch.cat(features)

In [ ]:
image_folderpath = '/media/ubuntu/sda/neuropixels/natural_movie/natural_movie_1_frame'
image_list = []
for i in range(900):
    image = Image.open(os.path.join(image_folderpath, f"frame_{i}.png")).convert("L")
    image_list.append(image)

image_features = image_encoder(image_list, vlmodel, preprocess_train, 20)


In [13]:
image_features = pd.DataFrame(image_features.numpy())

In [18]:
image_features = image_features.iloc[2:, :]

In [19]:
image_features.shape

(898, 1024)

In [20]:
image_features.index = list(range(898))

In [21]:
image_features.to_csv("/media/ubuntu/sda/neuropixels/natural_movie/natural_movie_1_frame_features.csv")